# Tutorial

This is a tutorial on how to add new safety algorithms and new models to the benchmark.


### Add a Safety Algorithm

First, we show how to add a new safety algorithm -- **Naive**.

Safety algorithms are located at src/agents. All algorithms are subclass of **MobileAgent**.

To create a new algorithm. First step, create a new file src/agents/Naive.py and import all the pacakges you need.

In [ ]:
from .MobileAgent import MobileAgent
import numpy as np
from numpy.matlib import repmat
from numpy import zeros, eye, ones, sqrt, asscalar
from cvxopt import solvers, matrix

Then create a class **Naive** follow the template of **MobileAgent**.

Initialize the class.

In [ ]:

class Naive(MobileAgent):

    def __init__(self):
        
        MobileAgent.__init__(self)
        d_min = 2

    def calc_control_input(self, dT, goal, fx, fu, Xr, Xh, dot_Xr, dot_Xh, Mr, Mh, p_Mr_p_Xr, p_Mh_p_Xh, u0, min_u, max_u):
        pass

*clac_control_input* function returns a control input based on internal state and obstacle information

    Args:
        dT (float): dT
        goal (ndarray): [x; y; z; 0; 0; 0]
        fx (ndarray): transition matrix
        fu (ndarray): transition matrix
        Xr (ndarray): estimate state of robot
        Xh (ndarray): estimate state of obstacle
        dot_Xr (ndarray): estimate gradient of robot state
        dot_Xh (ndarray): estimate gradient of obstacle state
        Mr (ndarray): cartesian position and velocity of the nearest point from robot to obstacle
        Mh (ndarray): cartesian position and velocity of the nearest point from obstacle to robot
        p_Mr_p_Xr (ndarray): derivative of robot cartesian state to internal state
        p_Mh_p_Xh (ndarray): derivative of obstacle cartesian state to internal state
        u0 (ndarray): reference control input, this is the default control input if there is no obstacle
        min_u (ndarray): control input saturation
        max_u (ndarray): control input saturation

Fill up the function with a naive policy:
If the distance between agent and obstacle is less than d_min, then give a control input towards opposite of the obstacle. Otherwise, return reference control input.

In [ ]:
    def calc_control_input(self, dT, goal, fx, fu, Xr, Xh, dot_Xr, dot_Xh, Mr, Mh, p_Mr_p_Xr, p_Mh_p_Xh, u0, min_u, max_u):
        
        dim = np.shape(Mr)[0] // 2 #dimesion of space, usually 2 or 3.
        p_idx = np.arange(dim) # Mr[p_idx] if the cartesian position of robot
        v_idx = p_idx + dim # Mr[p_idx] if the cartesian velocity of robot

        d = np.linalg.norm(Mr[p_idx] - Mh[p_idx]) # relative distance
        
        dM = Mr - Mh
        dp = dM[p_idx,0]  # relative position

        if d > self.d_min:
            return u0
        
        u = fu.T * p_Mr_p_Xr.T * (-dp) # control input towards opposite of the obstacle direction

        return u

### Add a Model

We show how we add the **Ball** robot model.

Models are located at src/models. All models are subclass of **KinematicModel**.

To create a new model, first import your required packages and create a class follow the template of **KinematicModel**.

There are two parts of codes in the **KinematicModel**. They are control part and graphics part seperately.

You just need to fill up the functions after the comment in each part:
```python
# The following functions are required to fill up for new models.
```

Here we listed all the functions required to be fill up.

In [ ]:
from .KinematicModel import KinematicModel
import numpy as np
from numpy.matlib import repmat
from numpy import zeros, eye, ones, matrix

from numpy import cos, sin, arccos, sqrt, pi, arctan2
from panda3d.core import *
from direct.gui.DirectGui import *

class Ball(KinematicModel):

    def __init__(self, agent, dT, auto=True, init_state = [-5,-5,0,0,0,0]):
        pass

    def init_x(self, init_state):
        """
        init state x
        """
        pass
    def set_saturation(self):
        """
        Set min and max cut off for state x and control u.
        """
        pass
    def get_P(self):
        """
        Return position in the Cartisian space.
        """
        pass
    def get_V(self):
        """
        Return velocity in the Cartisian space.
        """
        pass
    def set_P(self, p):
        """
        Set position in the Cartisian space.

        Args:
            p (ndarray): position
        """
        pass
    def set_V(self, v):
        """
        Set velocity in the Cartisian space

        Args:
            v (ndarray): velocity
        """
        pass
    
    def A(self):
        """
        Transition matrix A as explained in the class definition.
        """
        pass
    def B(self):
        """
        Transition matrix B as explained in the class definition.
        """
        pass
    def get_closest_X(self, Mh):
        """
        Update the corresponding state of the nearest cartesian point on self to obstacle. 
        
        Args:
            Mh (ndarray): 6*1 array, cartesian postion and velocity of the obstacle.
        """
        pass
    def p_M_p_X(self): # p closest point p X
        """ dM / dX, the derivative of the nearest cartesian point to robot state.
        """
        pass
    def estimate_state(self):
        """
        State estimater caller.
        """
        pass
    def u_ref(self):
        """
        Reference control input.
        """
        pass
    
    def load_model(self, render, loader, color=[0.1, 0.5, 0.8, 0.8], scale=0.5):
        """
        Load the 3d model to be shown in the GUI
        
        Args:
            render : panda3d component
            loader : panda3d component
            color (list): RGB and alpha
            scale (float): scale to zoom the loaded 3d model.
        """
        self.color = color
        self.render = render

    def redraw_model(self):
        """
        Refresh the position of the robot model and goal model in the GUI.
        """
        pass

The filled **Ball** class is like

In [ ]:

class Ball(KinematicModel):

    """
    This the 2D ball model. The robot can move to any direction on the plane.
    """
    
    def __init__(self, agent, dT, auto=True, init_state = [-5,-5,0,0]):

        self.max_v = 2
        self.max_a = 4

        KinematicModel.__init__(self, init_state, agent, dT, auto, is_2D = True)
        self.goals[2,:] = zeros(100)
        self.goal = np.vstack(self.goals[:,0])

    def init_x(self, init_state):
        x = np.vstack(init_state)
        self.x = matrix(zeros((4,1)))
        self.set_P(x[[0,1]])
        self.set_V(x[[2,3]])
        
        self.u = matrix(zeros((2,1)))
        self.m = self.get_PV()

    def set_saturation(self):
        self.max_u =  matrix([[self.max_a], [self.max_a]])
        self.min_u = -matrix([[self.max_a], [self.max_a]])
        self.max_x =  matrix([np.inf, np.inf, self.max_v, self.max_v]).T
        self.min_x = -matrix([np.inf, np.inf, self.max_v, self.max_v]).T

    def estimate_state(self, obstacle):
        self.kalman_estimate_state()
 
    def get_P(self):
        return np.vstack([self.x[[0,1]], 0]);
    
    def get_V(self):
        return np.vstack([self.x[[2,3]], 0]);

    def get_PV(self):
        return np.vstack([self.get_P(), self.get_V()])

    def get_closest_X(self, Xh):
        self.m = self.get_PV()
        return self.m

    def set_P(self, p):
        self.x[0,0] = p[0];
        self.x[1,0] = p[1];

    def set_V(self, v):
        self.x[2,0] = v[0];
        self.x[3,0] = v[1];

    def A(self):
        A = matrix(eye(4,4));
        A[0,2] = self.dT;
        A[1,3] = self.dT;
        return A;

    def B(self):
        B = matrix(zeros((4,2)));
        B[0,0] = self.dT**2/2;
        B[1,1] = self.dT**2/2;
        B[2,0] = self.dT;
        B[3,1] = self.dT;
        return B;

    def p_M_p_X(self): # p closest point p X
        ret = zeros((6,4))
        ret[0,0] = 1
        ret[1,1] = 1
        ret[3,2] = 1
        ret[4,3] = 1
        return ret

    def u_ref(self):
        K = matrix([[1,0,2,0],[0,1,0,2]]);
        dp = self.observe(self.goal[[0,1]] - self.m[[0,1]]);
        dis = np.linalg.norm(dp);
        v = self.observe(self.m[[3,4]]);

        if dis > 2:
            u0 = dp / max(abs(dp)) * self.max_a;
        else:
            u0 = - K*(self.m[[0,1,3,4]] - self.goal[[0,1,3,4]]);
        return u0;



    def add_BB8(self, pos, color, scale=0.5):
        ret = loader.loadModel("resource/BB8/bb8.obj")
        ret.reparentTo(self.render)
        ret.setTransparency(TransparencyAttrib.MAlpha)
        ret.setColor(color[0], color[1], color[2], color[3])
        ret.setP(90)
        ret.setScale(scale / 50)
        ret.setPos(pos[0], pos[1], pos[2]-0.5)
        
        body_diff = loader.loadTexture('resource/BB8/Body diff MAP.jpg')
        tbd = TextureStage('body diff')
        ret.setTexture(tbd, body_diff)
        
        pivot = render.attachNewNode("ball")
        pivot.setPos(pos[0], pos[1], pos[2]) # Set location of pivot point
        ret.wrtReparentTo(pivot) # Preserve absolute position

        return pivot;

    def load_model(self, render, loader, color=[0.1, 0.5, 0.8, 0.8], scale=0.5):
        KinematicModel.load_model(self, render, loader, color, scale)
        self.agent_model = self.add_BB8(list(self.get_P()[:,0]), color, scale);
        self.goal_model = self.add_sphere([self.goal[0], self.goal[1],0], color[:-1]+[0.5], scale);

    
    def redraw_model(self):
        self.agent_model.setPos(self.get_P()[0], self.get_P()[1], 0)
        self.goal_model.setPos(self.goal[0], self.goal[1], 0)

There is a model_auxiliary function in the superclass. This function is used to draw auxiliary graphics during debug. You can add some codes to this function and turn on the auxiliary option.

In the **Ball** model, we draw the speed direction and control direction, as well as the valid control half space.

In [ ]:
    def model_auxiliary(self):
        if not hasattr(self, 'robot_v'):
            [self.robot_v, self.robot_u] = self.draw_movement(list(self.get_PV()[:,0]), list(self.u[:,0])+[0]);
        else:
            self.move_seg(self.robot_v, list(self.get_P()[:,0]), list(self.get_V()[:,0]))
            self.move_seg(self.robot_u, list(self.get_P()[:,0]), list(self.u[:,0])+[0])
            self.remove_half_planes()
            self.add_half_planes()
